# ANU ASTR4004 Week 9

Author: Dr Sven Buder (sven.buder@anu.edu.au)

In [ ]:
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

import numpy as np
import matplotlib.pyplot as plt

#initialize a random number generator
rng = np.random.default_rng()

## Model Selection and Validation

You have data, you've estimated and re-estimated your uncertainties using the empirical methods described above, and now you want to fit a line to it. But is a line the right thing? Maybe there are two separate relations and some break point? Surely a curve would be better? Maybe there are two different physical models that could describe what you're seeing?

Model selection is a complicated topic, of which I'm only going to scratch the surface in order to highlight that such methods exist, provide some examples, and demonstrate a few simple implementations of them in practice. More information can be found in, for example, Ding et al. 2019 (https://arxiv.org/abs/1810.09583), and a miriad of additional tools for model selection can be found in https://scikit-learn.org/.

In many cases, the _very first question_ you should be asking yourself is if there is a physical reason to adopt a particular model vs. another. 

In [ ]:
# first let's generate some data
N_points = 30 #number of points
noise_scale = 0.1 #scaling for noise
true_order = 3
poly_coeffs = rng.uniform(low=-1, high=1, size=true_order+1)
#poly_coeffs = [0.5, -0.33, 0.137, 0.5] #some random polynomial coefficients

x = rng.uniform(size=N_points)*2 - 1.
y = np.polynomial.polynomial.polyval(x, poly_coeffs)
yerr = 0.05 + noise_scale*rng.random(N_points)
y += yerr*rng.standard_normal(size=N_points)

#plot the polynomial and data
fig, ax = plt.subplots()
rx = np.linspace(-1,1,100)
ax.plot(rx, np.polynomial.polynomial.polyval(rx, poly_coeffs), 'k-', label='fiducial model')
ax.errorbar(x, y, yerr=yerr, fmt="o", color='0.7', label='mock data')
ax.legend()
plt.xlabel('x value')
plt.ylabel('y value')
plt.title('Mock data drawn from polynomial')
plt.show()

### AIC and BIC
Akaike Information Criteria (AIC) and Bayesian Information Criteria (BIC) are two different ways to compare models of varying complexity, accounting for both changes in likelihood and penalising the inclusion of additional parameters (more complex models). 

For a given model, the AIC is defined as

\begin{equation}
\mathrm{AIC} = 2k - 2\ln\left(\hat{L}\right),
\end{equation}

where $k$ is the number of free parameters in the model and $\hat{L}$ is the maximum value of the likelihood function. BIC has a similar form but with a different penalty for the number of parameters:

\begin{equation}
\mathrm{BIC} = k\ln(n) - 2\ln\left(\hat{L}\right).
\end{equation}

The additional parameter $n$ is the number of data points used to constrain the model.

The different forms of AIC and BIC come from different assumptions with regard to priors in their formulation, and they have fundamentally different goals. Selection via AIC aims to minimise loss for predictive purposes, while BIC aims to identify the best model for inferrence purposes. In general the smaller penalty for complex models in AIC tends to favour a larger number of parameters.

Suffice it to say there's a lot to unpack between these two that is beyond the scope of this course. 

In [ ]:
#Let's have a look at how BIC and AIC compare for a simple case of polynomial fitting here
orders = np.arange(1,10)

fig, ax = plt.subplots(len(orders), figsize=(8,2*len(orders)), sharex=True)
rx = np.linspace(-1,1,100)

store_aic = np.zeros(len(orders))
store_bic = np.zeros(len(orders))

ylow, yhigh = y.min()-0.05*y.ptp(), y.max()+0.5*y.ptp()

for ii, polyorder in enumerate(orders):

    #estimate best-fit coefficients for this polynomial order
    coeffs = np.polynomial.polynomial.polyfit(x, y, polyorder, w=1./yerr)

    #estimate log(L) from best fit, assuming Gaussian uncertainties
    diff = (y-np.polynomial.polynomial.polyval(x, coeffs))**2
    sigma2 = yerr**2
    logl = -0.5*np.sum(diff/sigma2 + np.log(sigma2))

    #compute AIC and BIC
    aic = 2*(polyorder+1) - 2*logl
    bic = (polyorder+1)*np.log(len(x)) - 2*logl
    
    #save AIC/BIC for plotting later
    store_aic[ii] = aic
    store_bic[ii] = bic
    
    #plot panels showing data and best fit in ax
    ax[ii].errorbar(x,y,yerr=yerr, fmt='o', color='0.7', ms=3)
    ax[ii].plot(rx, np.polynomial.polynomial.polyval(rx, coeffs), 'k-')
    
    ax[ii].axis([-1,1,0.1,0.9])
    
    ax[ii].text(0.02, 0.05, f'Polynomial Order: {polyorder}; AIC: {aic:.0f}; BIC: {bic:.0f}', 
                size=12, transform=ax[ii].transAxes)

    ax[ii].set_ylabel('y value')

    ax[ii].axis([-1,1,ylow, yhigh])

ax[-1].set_xlabel('x value')
plt.show()

#plot showing AIC/BIC vs. polyorder
fig2, ax2 = plt.subplots()
ax2.plot(orders, store_aic, '-', color='C0', label='AIC')
ax2.plot(orders, store_bic, '-.', color='C1', label='BIC')

#get some ranges for the axes
aymin = np.minimum(store_aic.min(), store_bic.min())
aymax = np.maximum(store_aic.max(), store_bic.max())
ax2.plot([true_order,true_order],[aymin-5,aymax+5], 'r-', zorder=0.9)

ax2.legend()
plt.ylim(aymin-5, aymax+5)
ax2.set_xlabel('polynomial order')
ax2.set_ylabel('AIC/BIC')
plt.show()

### Leave-one-out cross validation (LOOCV)

Cross validation as a general tool involves splitting the sample into "training" and "testing" subsamples. The model is then fitted or trained using the training subsample, and its performance then evaluated by comparing predictions with the testing sample. In this way, it is considered an out-of-sample test (for comparison, an example of an in-sample leave-one-out test is jacknifing). Its behaviour is asymptotically equivalent to AIC (in the large sample limit), however it involves fewer assumptions about how additional parameters should be penalised. CV in general is a common approach in the context of machine learning, and so it's worth being familiar with the idea even if you're not likely to use it much.

LOOCV involves iterating the modelling procedure over the sample data, each time excluding one point, and using that to evaluate the out-of-sample performance. At each iteration one stores the mean square error for the left-out sample, and the final metric is the mean of those mean square error values.

LOOCV is a particular limit of *k*-fold cross validation, which involves dividing the data into *k* different subsets for training/testing, with *k* equal to the number of samples. For larger samples, LOOCV can be inefficient and adopting *k*-fold validation with *k*=5 or 10 can be useful.

In [ ]:
#Let's take our polynomial example again
orders = np.arange(1,6)

store_mse = np.zeros(len(orders))

# Let's do leave-one-out cross-validation for each polynomial order
for ii, polyorder in enumerate(orders):
    mean_off = 0.

    # Not let's leave one point out at a time at position jj
    for jj in range(len(x)):
        coeffs = np.polynomial.polynomial.polyfit(np.r_[x[:jj],x[jj+1:]],
                                                  np.r_[y[:jj],y[jj+1:]],
                                                  polyorder)
        #use the fitted coefficients to measure the offset for the excluded point
        mean_off += ((y[jj]-np.polynomial.polynomial.polyval(x[jj], coeffs))/yerr[jj])**2
    store_mse[ii] = mean_off / (len(x)-1)

fig, ax = plt.subplots()
ax.plot(orders, store_mse)

ax.axvline(true_order, color='r')
ax.set_ylabel('RMSE')
ax.set_xlabel('Polynomial order')
plt.show()


## Time Series

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing

We'll generate some sample time series data for the examples below:

In [ ]:
# Generating sample time series data
np.random.seed(42)
time_index = pd.date_range(start='2020-01-01', periods=100, freq='M')
data = pd.Series(np.random.randn(100).cumsum() + 50, index=time_index)

# Plot the time series data
plt.plot(data, label='Original Data')
plt.title('Sample Time Series Data')
plt.legend()
plt.show()

### Simple Exponential Smoothing (SES)

Simple Exponential Smoothing is a time series forecasting method for univariate data without any trend or seasonality. It only requires one smoothing parameter (α).

In [ ]:
# Simple Exponential Smoothing
ses_model = SimpleExpSmoothing(data).fit(smoothing_level=0.2, optimized=False)
ses_forecast = ses_model.forecast(12)

# Plot the original data and the SES forecast
plt.plot(data, label='Original Data')
plt.plot(ses_forecast, label='SES Forecast', color='red')
plt.title('Simple Exponential Smoothing')
plt.legend()
plt.show()

In this example, we've set the smoothing parameter (α) to 0.2. The fit method trains the model, and forecast is used to generate future predictions.

### Holt's Linear Method

Holt's Linear Method extends SES to capture both level and trend. It is useful when the data exhibits a linear trend.

In [ ]:
# Holt's Linear Method
holt_model = ExponentialSmoothing(data, trend='add').fit()
holt_forecast = holt_model.forecast(12)

# Plot the original data and the Holt's Linear forecast
plt.plot(data, label='Original Data')
plt.plot(holt_forecast, label="Holt's Linear Forecast", color='red')
plt.title("Holt's Linear Method")
plt.legend()
plt.show()

Here, the trend='add' option tells the model to include a linear trend.

### Additive Damped Trend Method

The additive damped trend method is a variation of Holt’s linear method where the trend is damped over time, meaning that the trend decreases in impact as the forecast extends into the future.

In [ ]:
# Additive Damped Trend Method
damped_model = ExponentialSmoothing(data, trend='add', damped_trend=True).fit()
damped_forecast = damped_model.forecast(12)

# Plot the original data and the damped trend forecast
plt.plot(data, label='Original Data')
plt.plot(damped_forecast, label="Additive Damped Trend Forecast", color='red')
plt.title("Additive Damped Trend Method")
plt.legend()
plt.show()

The damped_trend=True option applies damping to the trend component.

### Holt-Winters Method (Triple Exponential Smoothing)

The Holt-Winters method adds a seasonal component to Holt's linear method. It is especially useful for data with both a trend and seasonality.

In [ ]:
# Holt-Winters Method
hw_model = ExponentialSmoothing(data, trend='add', seasonal='add', seasonal_periods=12).fit()
hw_forecast = hw_model.forecast(12)

# Plot the original data and the Holt-Winters forecast
plt.plot(data, label='Original Data')
plt.plot(hw_forecast, label="Holt-Winters Forecast", color='red')
plt.title("Holt-Winters (Triple Exponential Smoothing)")
plt.legend()
plt.show()

The seasonal='add' option introduces seasonality, and seasonal_periods=12 assumes a yearly seasonal pattern (since our data is monthly).

### In Practice?

Let's look at some time-series data:

#### SES for ASX200

The ASX A200 index represents a wide range of industries in Australia, and the index’s movements can be influenced by various market conditions such as economic policies, global events, and company earnings.

Given the random noise and short-term fluctuations, **Simple Exponential Smoothing (SES)** is a helpful tool for identifying the long-term level of the index by smoothing out day-to-day variations.

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd

import warnings

warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)   # ValueWarning subclasses UserWarning

start_training = '2020-01-01'
end_training = '2024-09-15'

# start_predicting = '2024-09-01'
# end_predicting = '2024-10-01'

start_predicting = '2024-09-16'
end_predicting = '2025-10-01'

# Fetch ASX A200 index data (ASX:A200 ticker)
asx_a200 = yf.download('^AXJO', start=start_training, end=end_training, auto_adjust=True)
asx_a200_actual = yf.download('^AXJO', start=start_predicting, end=end_predicting, auto_adjust=True)

In [ ]:
asx_a200

In [ ]:
# Let's try to predict the Close Price
data_train = asx_a200['Close']
data_test = asx_a200_actual['Close']

In [ ]:
# Forecast the same number of days as in data_test
forecast_length = len(data_test)
forecast_index = data_test.index

# Let's try Simple Exponential Smoothing first
ses_model = SimpleExpSmoothing(data_train).fit(smoothing_level=0.2, optimized=False)
ses_forecast = ses_model.forecast(forecast_length)
ses_forecast.index = data_test.index#pd.date_range(start=start_predicting, periods=forecast_length, freq='B')

# Apply Holt's ExponentialSmoothing Linear Trend method
holt_model = ExponentialSmoothing(data_train, trend='add').fit()
holt_forecast = holt_model.forecast(forecast_length)
holt_forecast.index = forecast_index

hw_model = ExponentialSmoothing(data_train, trend='add', seasonal='add', seasonal_periods=5*12).fit()
hw_forecast_seasonal = hw_model.forecast(forecast_length)
hw_forecast_seasonal.index = forecast_index

# Plot the stock data, SES forecast, and actual data
plt.figure(figsize=(10,4))
plt.plot(data_train, label='ASX A200 Index Price', color = 'k')
plt.plot(ses_forecast, label='SES Forecast', color='C2')
plt.plot(holt_forecast, label="Holt's ExponentialSmoothing Linear Forecast", color='C6')
plt.plot(hw_forecast_seasonal, label="Holt-Winters Seasonal Forecast (12 weeks)", color='C4')
plt.plot(data_test, label='ASX A200 Index Price Actual', color='C3')
plt.xlim(pd.Timestamp(start_training), pd.Timestamp('2025-10-01'))
plt.title('Simple Exponential Smoothing on ASX A200 Index (2020 to 2025)')
plt.legend()
plt.show()

#### Holt's Linear Trend Method on Australian Retail Sales**

Retail sales often exhibit both a level and a trend over time, reflecting growth or decline in the economy. Holt’s Linear Trend method can be applied to model both the level and trend of retail sales data.

You can download Australian retail sales data from the Australian Bureau of Statistics (ABS) or from public datasets like Kaggle.

For this tutorial, I have gone to the ABS website (https://www.abs.gov.au) and searched for "Retail Trade, Australia" and then downloaded Table 1 of "Retail turnover, by industry group"

In [ ]:
retail_sales_abs = pd.read_excel('data/abs_retail_sales.xlsx',1,skiprows=9)
retail_sales_abs['date'] = pd.to_datetime(retail_sales_abs['Series ID'])
retail_sales_abs.set_index('date', inplace=True)

# let's only look at the last 100 months
retail_sales_abs = retail_sales_abs[-100:]

# Let's only look at Food Retailing
retail_sales_column = 'A3348591K' # Turnover ;  Total (State) ;  Food retailing ;
retail_sales_name = 'Food retailing'

retail_sales_abs

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

# Apply Simple Exponential Smoothing (SES)
ses_model = SimpleExpSmoothing(retail_sales_abs[retail_sales_column]).fit(smoothing_level=0.2, optimized=False)
ses_forecast = ses_model.forecast(12)  # Forecasting 12 months ahead

# Plot the original data and the SES forecast
plt.figure(figsize=(10, 6))
plt.plot(retail_sales_abs[retail_sales_column], label=retail_sales_name)
plt.plot(ses_forecast, label='SES Forecast', color='red')
plt.title('Simple Exponential Smoothing on ABS Retail Sales Data')
plt.legend()
plt.show()

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Apply Holt's Linear Trend method
holt_model = ExponentialSmoothing(retail_sales_abs[retail_sales_column], trend='add').fit()
holt_forecast = holt_model.forecast(12)  # Forecasting 12 months ahead

# Plot the original data and Holt's Linear forecast
plt.figure(figsize=(10, 6))
plt.plot(retail_sales_abs[retail_sales_column], label=retail_sales_name)
plt.plot(holt_forecast, label="Holt's Linear Forecast", color='red')
plt.title("Holt's Linear Trend Method on ABS Retail Sales Data")
plt.legend()
plt.show()

In [ ]:
# Apply Additive Damped Trend method
damped_model = ExponentialSmoothing(retail_sales_abs[retail_sales_column], trend='add', damped_trend=True).fit()
damped_forecast = damped_model.forecast(12)

# Plot the original data and the damped trend forecast
plt.figure(figsize=(10, 6))
plt.plot(retail_sales_abs[retail_sales_column], label=retail_sales_name)
plt.plot(damped_forecast, label="Additive Damped Trend Forecast", color='red')
plt.title("Additive Damped Trend Method on ABS Retail Sales Data")
plt.legend()
plt.show()

In [ ]:
# Apply Holt-Winters (Triple Exponential Smoothing)
hw_model = ExponentialSmoothing(retail_sales_abs[retail_sales_column], trend='add', seasonal='add', seasonal_periods=12).fit()
hw_forecast = hw_model.forecast(12)

# Plot the original data and the Holt-Winters forecast
plt.figure(figsize=(10, 6))
plt.plot(retail_sales_abs[retail_sales_column], label=retail_sales_name)
plt.plot(hw_forecast, label="Holt-Winters Forecast", color='red')
plt.title("Holt-Winters Method on ABS Retail Sales Data")
plt.legend()
plt.show()